# First experiment

In this notebook I am going to try and make a model forget 1 output class using MAAV differences. This will also be where I do some debugging of a few modules that I have built, but will probably remove those parts because they are not very interesting. 

I can look at the output values, and write up some thoughts, might test a couple of different classes. MNIST is interesting to use here because I can see very clear similarities and differences. 7 is similar 1, and very different to 8 for example. 

There are some of my notes here, don't worry about that, anything that I have worth saying is in the blog.

In [ ]:
import numpy as np
import cupy as cp

# This is needed to load the MNIST files for training and testing
import pickle

# These are the Sparana objects needed to build a model
from sparana.model import model
from sparana.layers import full_relu_layer
from sparana.layers import full_linear_layer
from sparana.optimizer import adam_optimizer
from sparana.data_loader import split_loader

# This is not needed to train and test a simple model, but I am going to demonstrate it here too
from sparana.saver import model_saver

# These are the extra imports mentioned above
from sparana.lobotomizer import lobotomizer
import os

# Put your own path in here
path = 'c:/users/jim/tensorflowtrials'

# This initializes the model object, the 2 things that are required are the input size, and a list of layer objects.
mymodel = model(input_size = 784, 
                # These are the layers input as a list, the final layer size is the number of classes the model will have.
                layers = [full_relu_layer(size = 1000), 
                          full_relu_layer(size = 800),
                          full_relu_layer(size = 400),
                          full_linear_layer(size = 10)],
                # This is set automatically, but I keep it in here as a demonstration
                comp_type = 'GPU')

# Initialize the weights here, after this randomly generated matrices are now in the GPU memory
mymodel.initialize_weights('Xavier', bias_constant = 0.1)

# Initialize the Adam optimizer, the associated matrices are now in GPU memory
opt = adam_optimizer(mymodel, 0.0001, epsilon = 0.001)

#Initialize the saver
mysaver = model_saver(mymodel)

# Split loader

This is the first bit that is different to what I have done before, not going into any detail here. Something like this will need to be built for any different application. This just splits the MNIST dataset into the main data, and the target data. 


In [ ]:
myloader = split_loader(pickle.load(open('MNIST_train_images.p', 'rb')),
                        pickle.load(open('MNIST_train_labels.p', 'rb')),
                        pickle.load(open('MNIST_test_images.p', 'rb')),
                        pickle.load(open('MNIST_test_labels.p', 'rb')),
                        remove_classes = [8],
                        maintain_classes = True)
# Load the model from pickle file.
mysaver.load_model('layers_and_pruning_100000_steps.p')
print(mymodel.get_accuracy(myloader.test_data(), myloader.test_labels()))
mysaver.store_model()

# Prune the model

Here I load the data and get the MAAV differences. I found that 500 datapoints was enough here. For a lot of cases you would probably just work with what you have got. 


In [ ]:
# Initialize the lobotomizer

lobo = lobotomizer(mymodel)

# Load the data to be passed through

main_data, main_labels = myloader.random_minibatch(500)
removed_data, removed_labels = myloader.removed_minibatch(500)

# This is the command that does the work

lobo.MAAV_difference(main_data, removed_data)

# This is the hack because I have not fixed prune_bigget() yet.

for i in lobo._weight_stats:
    i.data = -i.data

In [ ]:
main_acc = []
target_acc = []
parameters_removed = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 
                      7000, 8000,9000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000,
                      50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 
                      100000, 110000, 120000, 130000, 140000, 150000, 160000]
data, labels = myloader.removed_test_set()
for i in parameters_removed:
    lobo.prune_smallest(layers = [0, 0, i], print_stats = True, zero_stats = False)
    # The zero stats needs to be done here because I am inverting the vales
    main_acc.append(mymodel.get_accuracy(myloader.test_data(), myloader.test_labels()))
    target_acc.append(mymodel.get_accuracy(data, labels))
print(main_acc)
print(target_acc)

I have just cut and paste these into the arrays here. Sometimes I like to do this instead of automating it because it forces me to look at, and think about the data for 10 minutes. 

# Pruning layer 3

parameters_removed = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 110000, 120000, 130000, 140000, 150000, 160000]

main_acc = [0.9858, 0.9862, 0.9857, 0.9858, 0.9858, 0.986, 0.9862, 0.986, 0.9855, 0.9859, 0.986, 0.9859, 0.9859, 0.9862, 0.9863, 0.986, 0.9859, 0.9859, 0.9856, 0.9854, 0.9852, 0.9847, 0.9847, 0.9842, 0.984, 0.9846, 0.9843, 0.9846, 0.9846, 0.9846, 0.9846, 0.9844, 0.9846, 0.9843, 0.9839, 0.9839, 0.9839, 0.9837, 0.9839, 0.9835]

target_acc = [0.9815, 0.9723, 0.9692, 0.9579, 0.96, 0.9579, 0.961, 0.96, 0.9548, 0.9476, 0.9322, 0.9138, 0.9148 0.923, 0.9199, 0.9076, 0.8984, 0.8655, 0.8265, 0.8234, 0.7485, 0.7043, 0.6643, 0.537, 0.5719, 0.578, 0.4979, 0.4867, 0.423, 0.4641, 0.4271, 0.4415, 0.424, 0.4569, 0.4384, 0.4374, 0.4435, 0.384, 0.4754, 0.4569]

# Pruning layer 2

parameters removed = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 110000, 120000, 130000, 140000, 150000, 160000]

main_acc = [0.9851, 0.9858, 0.986, 0.9861, 0.9861, 0.9861, 0.9857, 0.986, 0.9857, 0.9859, 0.9858, 0.9858, 0.9853, 0.9853, 0.9853, 0.9853, 0.9847, 0.9851, 0.984, 0.9827, 0.9829, 0.982, 0.9813, 0.9806, 9804, 0.9784, 0.9774, 0.976, 0.9751, 0.9736, 0.9722, 0.9702, 0.9679, 0.9684, 0.9651, 0.9621, 0.9586, 0.9596, 0.9566, 0.9547]

target_acc = [0.9815, 0.9815, 0.9756, 0.9756, 0.9717, 0.9688, 0.9678, 0.9678, 0.962, 0.9542, 0.9533, 0.9416, 0.9379, 0.9357, 0.9095, 0.8949, 0.8151, 0.677, 0.6634, 0.6322, 0.6274, 0.4747, 0.4260, 0.4085, 0.4095, 0.3715, 0.3589, 0.3278, 0.3132, 0.3521, 0.3599, 0.3307, 0.3105, 0.2694, 0.247, 0.2091, 0.1906, 0.1916, 0.1595, 0.1507]

# Pruning both layers

parameters_removed = [0, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 12000, 15000, 20000, 25000, 30000, 50000]

main_acc = [0.9859, 0.9861, 0.9865, 0.9867, 0.9865, 0.9854, 0.9858, 0.986, 0.9862, 0.9864, 0.986, 0.986, 0.9859, 0.9858, 0.9856, 0.9853, 0.9855, 0.9859, 0.9855, 0.9858, 0.9855, 0.9849, 0.985, 0.9848, 0.9848, 0.9843, 0.9851, 0.9845, 0.9838, 0.9843, 0.98487, 0.983, 0.98, 0.9791, 0.9754, 0.9709, 0.946]

target_acc = [0.9824, 0.9776, 0.9737, 0.9684, 0.9571, 0.9416, 0.9260, 0.8579, 0.8171, 0.7577, 0.7645, 0.7344, 0.7071, 0.5307, 0.3978, 0.3638, 0.3891, 0.3861, 0.318, 0.2217, 0.1498, 0.1332, 0.1245, 0.1177, 0.1031, 0.0933, 0.0875, 0.0846, 0.0758, 0.068, 0.0632, 0.0583, 0.0437, 0.0301, 0.0116, 0.0116, 0.0087]

I got a bit bored of doing this every 100 parameters, so there is a jump. I am going to have to push this to the point that target accuracy is less than random. Have never really looked at how this sort of plot would look pruning just MAAVs or absolute values, maybe a slight sidetracked blog in this series? 

A limitation of this type of model is that it is not very deep, and I can only compare 1 layer to 2 layers. 

# Testing on 8s

Seems that I need to remove a lot more parameters to forget 8s. Are they more complicated? is there more variability in how they are written, maybe 7s are the simplest. This really needs to be its own blog to see what is going on. Too soon to tell if there is anything good or bad about this result, going to bed to think about this for a while. 

This does suggest that a fair ammount of testing and trial and error will be involved in the imaginary use case. I can sort of look at MNIST and see why an 8 is more complex than a 7. 

It could be down to the fact that the internal representation of a 7 is more compressed. It also takes less pixels to start with. 

I am still going to do the looking at outputs one first, build up a base of understanding. Look at other outputs next. Probably just prune both. I will need to design a single good plot here, the best ones to illustrate the differemce. Number of parameters, and point where main data acc starts to fall off too. This is just illustrative, I would expect this sort of thing to exist in other models too. 

I know that the drop in main data accuracy dropping is not the point where I need to stop, because retraining will recover the slight drops we see. I need to think of where to include this. 


# Plotting stuff

I have just cut and paste this from other notebooks, I comment out lines and change labels etc. 

In [ ]:
parameters_removed = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 110000, 120000, 130000, 140000, 150000, 160000]

main_acc = [0.9858, 0.9862, 0.9857, 0.9858, 0.9858, 0.986, 0.9862, 0.986, 0.9855, 0.9859, 0.986, 0.9859, 0.9859, 0.9862, 0.9863, 0.986, 0.9859, 0.9859, 0.9856, 0.9854, 0.9852, 0.9847, 0.9847, 0.9842, 0.984, 0.9846, 0.9843, 0.9846, 0.9846, 0.9846, 0.9846, 0.9844, 0.9846, 0.9843, 0.9839, 0.9839, 0.9839, 0.9837, 0.9839, 0.9835]

target_acc = [0.9815, 0.9723, 0.9692, 0.9579, 0.96, 0.9579, 0.961, 0.96, 0.9548, 0.9476, 0.9322, 0.9138, 0.9148, 0.923, 0.9199, 0.9076, 0.8984, 0.8655, 0.8265, 0.8234, 0.7485, 0.7043, 0.6643, 0.537, 0.5719, 0.578, 0.4979, 0.4867, 0.423, 0.4641, 0.4271, 0.4415, 0.424, 0.4569, 0.4384, 0.4374, 0.4435, 0.384, 0.4754, 0.4569]

import matplotlib.pyplot as plt
import pickle
import numpy as np


plt.plot(parameters_removed, main_acc, label = "Main Data Accuracy")
plt.plot(parameters_removed, target_acc, label = "Target Data Accuracy")

plt.title('MAAV Difference Pruning Layer 3')
plt.legend()
plt.grid()
plt.legend(loc=9, bbox_to_anchor=(0.5, -0.2), ncol=2)
#plt.ylim(0.98, 1)
#plt.xlim(0, 1500000)
plt.xlabel('Parameters Removed')
plt.ylabel('Accuracy')
#plt.gca().invert_xaxis()
#plt.xticks(ticks=prune_2, labels = prune_2)
plt.show()